In [2]:
import numpy as np
import pandas as pd
from scipy.integrate import ode as ode
from matplotlib import pyplot as plt
#import xlsxwriter
from scipy.optimize import curve_fit
import seaborn as sns
sns.set_style('whitegrid')
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

import os
#from model_vectorised_dg_extended import *
from ipynb.fs.full.model_vectorised_dg_extended import *
os.chdir(os.getcwd())

##### 1. Create 'data' folder in parent directory to save the simulation data there
##### 2. Create the two main subfolders: with internalisation/no internalisation -> all perturbations of the different processes are performed once with internalisation, once without internalisation
##### 3. Create sub-subfolders for the individual perturbed processess

In [9]:
if not os.path.exists('../data'):
    print(os.path.exists('../data'))
    os.makedirs('../data')

for intern_dirname in ['With_internalisation','No_internalisation']:
    path=os.path.join('../data/',intern_dirname)
    if not os.path.exists(path):
        os.makedirs(path)

perturbed_processes=['Koff_Kon_all','Phosphorylation_PM','Phosphorylation_IC','Phosphorylation_IC_PM','Phosphatase_PM','Phosphatase_IC','Phosphatase_IC_PM',
'Arrestin_association','Arrestin_dissociation','Internalisation']

for process_dirname in perturbed_processes:
    for intern_dirname in ['With_internalisation','No_internalisation']:
        path=os.path.join('../data/',intern_dirname,process_dirname,'')
        if intern_dirname=='No_internalisation' and process_dirname=='Internalisation':
            continue
        if not os.path.exists(path):
            os.makedirs(path)

### Perturbation of Kon-Koff all 

In [3]:
## Dictionary holding the numbers of the molecules in the model (for explanation, look up model-figure.pdf)
molecules_dict={'Arrestin-Receptor':[10,11,12,13,14,15],
                'Ligand':[1,3],
                'Arrestin': [2],
                'Receptor':[4]}


### Dictionary holding the name of the perturbed processes and the number of perturbed processes that they 
##  are corresponding to in the model (i.e. Kon value of L + R -> LR is K value number 1,5,9,..., check out model-figure.pdf for visual
## explanation)

### In each perturbation round two sets of processes are perturbed:
##  1. Kon-Koff values of the Ligand-Receptor binding
##  2. Other cellular processes (i.e. internalisation, phosphorylation, etc...)
##  By looping this dictionary each 

## 1. "Koff_Kon_perturbations" hold the perturbed values (kon_values,koff_values) but also the number of the Kon-Koff values 
#  in the model (kon_model_numbers,koff_model_numbers)
## 2. "Process_model_numbers" hold the model numbers for the respective cellular process 


perturbed_processes={'Koff_Kon_all':{'Koff_Kon_perturbations':{ 'kon_values':[0.03,0.3,3],
                                                                'kon_model_numbers':[1,5,9,13,17,21,51,55],
                                                                'koff_values':[0.0003,0.03,0.3],
                                                                'koff_model_numbers':[2,6,10,14,18,22,4,8,12,16,20,24,52,56,54,58]},
                                     'Process_model_numbers':[]},

                    'Phosphorylation_PM':{'Koff_Kon_perturbations':{'kon_values':[0.3],
                                                                    'kon_model_numbers':[1,5,9,13,17,21,51,55],
                                                                    'koff_values':[0.0003,0.03,0.3],
                                                                    'koff_model_numbers':[2,6,10,14,18,22,4,8,12,16,20,24,52,56,54,58]},
                                        'Process_model_numbers':[30]},

                    'Phosphorylation_IC':{'Koff_Kon_perturbations':{'kon_values':[0.3],
                                                                    'kon_model_numbers':[1,5,9,13,17,21,51,55],
                                                                    'koff_values':[0.0003,0.03,0.3],
                                                                    'koff_model_numbers':[2,6,10,14,18,22,4,8,12,16,20,24,52,56,54,58]},
                                        'Process_model_numbers':[50]},

                    'Phosphorylation_IC_PM':{'Koff_Kon_perturbations':{ 'kon_values':[0.3],
                                                                        'kon_model_numbers':[1,5,9,13,17,21,51,55],
                                                                        'koff_values':[0.0003,0.03,0.3],
                                                                        'koff_model_numbers':[2,6,10,14,18,22,4,8,12,16,20,24,52,56,54,58]},
                                            'Process_model_numbers':[50,30]},

                    'Phosphatase_PM':{'Koff_Kon_perturbations':{'kon_values':[0.3],
                                                                'kon_model_numbers':[1,5,9,13,17,21,51,55],
                                                                'koff_values':[0.0003,0.03,0.3],
                                                                'koff_model_numbers':[2,6,10,14,18,22,4,8,12,16,20,24,52,56,54,58]},
                                      'Process_model_numbers':[25,27,29]},

                    'Phosphatase_IC':{'Koff_Kon_perturbations':{'kon_values':[0.3],
                                                                'kon_model_numbers':[1,5,9,13,17,21,51,55],
                                                                'koff_values':[0.0003,0.03,0.3],
                                                                'koff_model_numbers':[2,6,10,14,18,22,4,8,12,16,20,24,52,56,54,58]},
                                      'Process_model_numbers': [47,48,49]},  

                    'Phosphatase_IC_PM':{'Koff_Kon_perturbations':{ 'kon_values':[0.3],
                                                                    'kon_model_numbers':[1,5,9,13,17,21,51,55],
                                                                    'koff_values':[0.0003,0.03,0.3],
                                                                    'koff_model_numbers':[2,6,10,14,18,22,4,8,12,16,20,24,52,56,54,58]},
                                        'Process_model_numbers': [25,27,29,47,48,49]},

                    'Arrestin_association':{'Koff_Kon_perturbations':{  'kon_values':[0.3],
                                                                        'kon_model_numbers':[1,5,9,13,17,21,51,55],
                                                                        'koff_values':[0.0003,0.03,0.3],
                                                                        'koff_model_numbers':[2,6,10,14,18,22,4,8,12,16,20,24,52,56,54,58]},
                                            'Process_model_numbers': [35,44,36,37,45,46]},

                    'Arrestin_dissociation':{'Koff_Kon_perturbations':{ 'kon_values':[0.3],
                                                                        'kon_model_numbers':[1,5,9,13,17,21,51,55],
                                                                        'koff_values':[0.0003,0.03,0.3],
                                                                        'koff_model_numbers':[2,6,10,14,18,22,4,8,12,16,20,24,52,56,54,58]},
                                             'Process_model_numbers': [32,33,41,42,34,43]},

                    'Internalisation':{'Koff_Kon_perturbations':{   'kon_values':[0.3],
                                                                    'kon_model_numbers':[1,5,9,13,17,21,51,55],
                                                                    'koff_values':[0.0003,0.03,0.3],
                                                                    'koff_model_numbers':[2,6,10,14,18,22,4,8,12,16,20,24,52,56,54,58]},
                                        'Process_model_numbers': [38, 39, 40]}}

                     
## Determine ligand concentrations              
ligand_ccs=(10 ** np.arange(-7, 2.1, 1))
internalisation_status=['With_internalisation','No_internalisation']

## Simulation time
simulation_time=0.3

## List of perturbation factors, that are used for the perturbations of cellular processes ->
## Each rate constant (contained by 'Process_model_numbers')in the cellular process to be perturbed gets multiplied 
# by this factor to change the process' speed
factor_list=[100,30,10,3,1,0.3,0.1,0.03,0.01]

## Calculate the number of all models
num_of_all_simulations=0
for key in perturbed_processes.keys():
    kon_num=len(perturbed_processes[key]['Koff_Kon_perturbations']['kon_values'])
    koff_num=len(perturbed_processes[key]['Koff_Kon_perturbations']['koff_values'])
    if len(perturbed_processes[key]['Process_model_numbers'])>0:
        if key=='Internalisation':
            num_of_sim_in_perturbation=kon_num*koff_num*len(factor_list)*len(ligand_ccs)
        if key!='Internalisation':
            num_of_sim_in_perturbation=kon_num*koff_num*len(factor_list)*len(ligand_ccs)*len(internalisation_status)
    elif len(perturbed_processes[key]['Process_model_numbers'])==0:
        num_of_sim_in_perturbation=kon_num*koff_num*len(ligand_ccs)*len(internalisation_status)
    num_of_all_simulations=num_of_all_simulations + num_of_sim_in_perturbation


## Function for running the simulation for a given set of parameters
def simulation(intern_status,process_name,ligand_ccs,simulation_time,sim_number,start_time,factor=None):
    ## Select the perturbed process out out of perturbed processes dictionary and initialise the Ko Koff values
    perturbed_process=perturbed_processes[process_name]
    kon_values=perturbed_process['Koff_Kon_perturbations']['kon_values']
    koff_values=perturbed_process['Koff_Kon_perturbations']['koff_values']
 
    ## Loop over Kon and Koff values
    for a,kon in enumerate(kon_values):
        ligands_dict={}
        for b,koff in enumerate(koff_values):
            ## Adjust ligand concentrations according to the changing Koff/Kon ratio (==Kd)-> changing ligand affinity needs changing
            # ligand concentrations to achieve same efficacy (if ligand affinity drops -> ligand concentration needs to 
            # achieve increase for same effect)
            Kd_ratio = (koff/ kon) / (0.0003 / 0.3)
            ligands = ligand_ccs * Kd_ratio
            #ligands_dict['koff_'+str(koff)]=10**(np.log10(ligands) - 6)

            ## Initialise list, that will hold the simulation data/ligand cc. for one Kon-Koff combination 
            data_list = []

            ## Loop over all the used ligand concentrations
            for ligand in ligands:
                ## Initialise model, and change the necessary rate constants, ligand concentrations
                model = VecModel(0.1)
                model.read('../model/toth_szalai_modified.txt')
                model.change_constant(perturbed_process['Koff_Kon_perturbations']['kon_model_numbers'], kon)
                model.change_constant(perturbed_process['Koff_Kon_perturbations']['koff_model_numbers'], koff)
                model.change_ccs(molecules_dict['Ligand'], ligand)

                ## In case of no internalisation, set internalisation rate constant to 0
                if intern_status=='No_internalisation':
                    model.change_constant(perturbed_processes['Internalisation']['Process_model_numbers'], 0)
        
                ## If there is a cellular process to ber perturbed, multiply the rate constants of the process by the perturbation factor
                if factor !=None:
                    model.multiply_constant_with_factor(perturbed_process['Process_model_numbers'], factor)

                ## Simulate the process for one constellation of parameters
                model.simulate(simulation_time)
                sim_number=sim_number+1
                elapsed_time=(time.time()-start_time)/3600
                remaining_time=((1/(sim_number/num_of_all_simulations))*elapsed_time-elapsed_time)
                print(round(100*sim_number/num_of_all_simulations,2),' % Done ','Elapsed time (h): ',round(elapsed_time,4),
                'Remaining time (h):',round(remaining_time,4),end="")
                print("\r", end="")

                ## Save concentrations of the molecules into a dataframe and add to list, where each element of the list corresponds to 
                ## the simulation result with one ligand cc. 
                ## Rows: time
                ## Columns: concentrations of molecules considered in the simulation
                data_list.append(model.convert_ccs_to_df())
            
            ## Save simulation result into an .xlsx file
            ## Each ligand concentration is saved into a different sheet in the same .xslx file
            dirname=os.path.join('../data/',intern_status,process_name,'')
            if factor !=None:
                fname=dirname+ '_'.join([process_name,str(factor),'koff',str(koff),'kon',str(kon)]) + '.xlsx'
            elif factor==None:
                fname=dirname+ '_'.join([process_name,'koff',str(koff),'kon',str(kon)]) + '.xlsx'
            model.save_as_xlsx_ligand_cc_per_sheet(fname,data_list,(np.log10(ligands) - 6))
        #model.save_as_xlsx('Model/Ligand concentrations/kon_'+str(kon_perturbation[a]) + '.xlsx',ligands_dict)
    #model.write('Time (s).xlsx',model.t)
    return(sim_number)


## Keep track of number of simulations and elapsed time
import time
sim_number=0
start_time = time.time()

## Loop over experimental settings and select parameters for the simulation
for intern_status in internalisation_status:
    for process_name in perturbed_processes.keys():

        ## In the case of no internalisation don't run perturbation of Internalisation process, as internalisation is set to 0.
        if intern_status=='No_internalisation' and process_name=='Internalisation':
            continue
        
        ## If there is a cellular process to be perturbed, loop over the perturbation factors that will be used for rate constant perturbation
        if len(perturbed_processes[process_name]['Process_model_numbers'])>0:
            for factor in factor_list:
                sim_number=simulation(intern_status,process_name,ligand_ccs,simulation_time,sim_number,start_time,factor=factor)
        
        ## If only Kon-Koff values are perturbed, don't loop over factors
        elif len(perturbed_processes[process_name]['Process_model_numbers'])==0:
            sim_number=simulation(intern_status,process_name,ligand_ccs,simulation_time,sim_number,start_time,factor=None)



In [4]:

#simulation perturbing the k values of 5 different processes ("Receptor phosphorylation","Receptor dephosphorylation","Arrestin association","Arrestin dissociation","Internalization")
#ec: extracellular, es:endosomal
constant_dict.update({"Receptor phosphorylation GRK":[26,28,30],"Receptor phosphorylation PKC":[31],"Receptor phosphorylation all":[26,28,30,31],\
                      "Receptor dephosphorylation ec":[25,27,29],"Receptor dephosphorylation es":[41,42,43],"Receptor dephosphorylation all":[25,27,29,41,42,43],\
                      "Arrestin association":[35,39],"Arrestin dissociation ec":[32,33,34],"Arrestin dissociation es": [37, 38, 40],\
                      "Arrestin dissociation all": [32,33,34,37, 38, 40],"Internalization":[36]})

koff_values=[0.01,0.001]
kon_values=[0.03,3]
arrestin_ccs=[15000,1000,1000]
receptor_ccs=[5000,5000,50]
ip3sensor_ccs=[0,6000,0]
factor_list=[0,1,1/30,1/10,1/3,3,10,30]
cellular_models=['Arrestin-Receptor overexpression','IP3-Receptor overexpression','Physiological system']

phosphorylation=[constant_dict["Receptor phosphorylation all"]]
dephosphorylation=[constant_dict["Receptor dephosphorylation all"]]
arrestin_association=[constant_dict["Arrestin association"]]
arrestin_dissociation=[constant_dict["Arrestin dissociation all"]]
internalization=[constant_dict["Internalization"]]

k_values_list=[phosphorylation,dephosphorylation,arrestin_association,arrestin_dissociation,internalization]
#k_names_list=[phosphorylation_names,dephosphorylation_names,arrestin_association_names,arrestin_dissociation_names,internalization_names]
k_values_directories=["Receptor phosphorylation","Receptor dephosphorylation","Arrestin association","Arrestin dissociation","Internalization"]

#simulation perturbing the k values of 5 different processes ("Receptor phosphorylation","Receptor dephosphorylation","Arrestin association","Arrestin dissociation","Internalization")
def k_perturbation_simulation(arrestin,receptor,ip3sensor,cellular_model,simulation_time,k_values,factor,k_values_dir):
    for k_value in k_values:
        for kon,koff in zip(range(len(kon_values)),range(len(koff_values))):
            ligands = (10 ** np.arange(-4, 1.1,1))
            Kd_ratio = (koff_values[koff] / kon_values[kon]) / (0.003 / 0.3)
            ligands = ligands * Kd_ratio
            data_list = []
            for ligand in ligands:
                model = VecModel(0.1)
                model.read('../models/toth_szalai_final.txt')
                model.change_ccs(molecules_dict['Arrestin'], arrestin)
                model.change_ccs(molecules_dict['Receptor'], receptor)
                model.change_ccs(molecules_dict['Sensor'], ip3sensor)
                model.change_ccs(molecules_dict['Ligand'], ligand)
                model.change_constant(constant_dict['Receptor-Ligand kon'], kon_values[kon])
                model.change_constant(constant_dict['Receptor-Ligand koff'], koff_values[koff])
                model.multiply_constant(k_value, factor)
                model.simulate(simulation_time)
                data_list.append(model.append_ccs())
            fname='Model/Simulation with K values perturbation/Simulated values/' + str(cellular_model) + '/'+k_values_dir+'/'+str(factor)+' koff '\
                              + str(koff_values[koff]) + ' - kon ' + str(kon_values[kon]) + '.xlsx'
            model.write_toexcel(fname, data_list, (np.log10(ligands) - 6))
"""
for k_values,k_values_dir,in zip(k_values_list,k_values_directories):
    for factor in factor_list:
        for arrestin,receptor,ip3sensor,cellular_model in zip(arrestin_ccs,receptor_ccs,ip3sensor_ccs,cellular_models):
            k_perturbation_simulation(arrestin,receptor,ip3sensor,cellular_model,simulation_time,k_values,factor,k_values_dir)

"""


NameError: name 'constant_dict' is not defined

In [ ]:

#Simulation with no internalization
koff_values=[0.0003,0.001,0.003,0.01,0.03]
kon_values=[0.03,0.3,3]
simulation_time=1200

def k_perturbation_simulation_no_internalization(arrestin,receptor,ip3sensor,cellular_model,simulation_time,k_values,factor,k_values_dir):
    for k_value in k_values:
        for kon in range(len(kon_values)):
            for koff in range(len(koff_values)):
                ligands = (10 ** np.arange(-7, 1.1,1))
                Kd_ratio = (koff_values[koff] / kon_values[kon]) / (0.003 / 0.3)
                ligands = ligands * Kd_ratio
                data_list = []
                for ligand in ligands:
                    model = VecModel(0.1)
                    model.read('../models/toth_szalai_final.txt')
                    model.change_ccs(molecules_dict['Arrestin'], arrestin)
                    model.change_ccs(molecules_dict['Receptor'], receptor)
                    model.change_ccs(molecules_dict['Sensor'], ip3sensor)
                    model.change_ccs(molecules_dict['Ligand'], ligand)
                    model.change_constant(constant_dict['Receptor-Ligand kon'], kon_values[kon])
                    model.change_constant(constant_dict['Receptor-Ligand koff'], koff_values[koff])
                    model.multiply_constant(k_value, factor)
                    model.simulate(simulation_time)
                    data_list.append(model.append_ccs())
                fname='Model/Simulation values/No internalization/'+str(cellular_model)+'/koff ' + str(koff_values[koff]) + " - kon " + str(kon_values[kon]) + ".xlsx"
                model.write_toexcel(fname, data_list, (np.log10(ligands) - 6))

for k_values,k_values_dir,in zip(k_values_list[-1:],k_values_directories[-1:]):
    for factor in factor_list[0:1]:
        for arrestin,receptor,ip3sensor,cellular_model in zip(arrestin_ccs,receptor_ccs,ip3sensor_ccs,cellular_models):
            k_perturbation_simulation_no_internalization(arrestin,receptor,ip3sensor,cellular_model,simulation_time,k_values,factor,k_values_dir)